In [3]:
### Illustration du biais de regularisation
### 26 septembre 2019
### Jeremy LHour

library("MASS")
library("glmnet")

# fonctions utilisateur
source("functions/DataSim.R") 

set.seed(12071990)
n = 2000
data = DataSim(n, p=20, Ry= .2, Intercept=F)

Warning message in c * gamma:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”Warning message in c * b:
“Recycling array of length 1 in array-vector arithmetic is deprecated.
  Use c() or as.vector() instead.
”

In [ ]:
PostLasso <- function(X,y,nopen){
    #'@param nopen indices des variables a ne pas penaliser
    phi = rep(1,ncol(X))
    phi[nopen] = 0
    cv.lasso = cv.glmnet(X,y, family="gaussian",alpha=1,penalty.factor=phi,nfolds=5)
    s.hat = (coef(cv.lasso) != 0)
    postlasso = lm(y ~ )
}